<a href="https://colab.research.google.com/github/TarekHasan011/Pattern-Recognition/blob/main/Pattern_Laboratory_Online_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading Data

In [ ]:
import pandas as pd
from pandas.api.types import is_string_dtype
data = pd.read_csv('hepatitis.csv', na_values='?')

for column in data.columns:
  if (is_string_dtype(data[column].dtype)):
    data[column] = data[column].str.strip()

# Spliting Data

In [ ]:
from sklearn.model_selection import train_test_split
X = data.loc[:, data.columns != 'Class']
y = data['Class']
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.1,stratify=y, random_state=5)


# Handling Missing Values

In [ ]:
from sklearn.impute import SimpleImputer
from pandas.api.types import is_string_dtype

si_train_X = pd.DataFrame()
si_test_X = pd.DataFrame()

for column in train_X.columns:
  if (is_string_dtype(train_X[column].dtype)):
    si = SimpleImputer(strategy='most_frequent')
  else:
    si = SimpleImputer(strategy='median')
  si.fit(train_X[[column]])
  si_train_X[column] = si.transform(train_X[[column]]).flatten()
  si_test_X[column] = si.transform(test_X[[column]]).flatten()

# Handling Text Features

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_y)
train_y = le.transform(train_y)
test_y = le.transform(test_y)

continuous_feature = ['AGE','BILIRUBIN','ALK PHOSPHATE','SGOT','ALBUMIN','PROTIME']
l_train_X = pd.DataFrame()
l_test_X = pd.DataFrame()

for column in train_X.columns:
  if column not in continuous_feature:
    le.fit(si_train_X[column])
    l_train_X[column] = le.transform(si_train_X[column])
    l_test_X[column] = le.transform(si_test_X[column])
  else:
    l_train_X[column] = si_train_X[column].copy()
    l_test_X[column] = si_test_X[column].copy()

# Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(l_train_X)
ss_train_X = ss.transform(l_train_X)
ss_test_X = ss.transform(l_test_X)

# Training Phase using SVM

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(ss_train_X,train_y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Test Phase

In [ ]:
predictions = svc.predict(ss_test_X)

# Performance Analysis

In [ ]:
from sklearn.metrics import accuracy_score, f1_score,recall_score, precision_score
accuracy = accuracy_score(test_y,predictions)
f1 = f1_score(test_y,predictions)
print(f'{accuracy} {f1}')

0.9375 0.962962962962963
